# Labeling by Document Similarity
We take a different approach to label the comments. In this approach, we use labeled comments and find similar comments to label them.

In [2]:
import os
import pandas as pd
import numpy as np
from top2vec import Top2Vec
import sys
import re
from pprint import pprint

c:\Users\doosti\.conda\envs\top2vec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading Data Sets

In [411]:

DATA_PATH = "C:/Users/doosti/Dropbox (Chapman)/Research/Research Projects/Fitness/Data/"
processed_file="processed_comments_122923.txt"
# use processed_comments_122923_fast.txt if newpreprocessed model is used (instead of newpreprocessed2)
comments_file="merged_comments.csv"
#labeled = "comments_activity_motives.csv"
#sim_labels = "comments_similarity_labels.csv"
labeled = "comments_similarity_labels.csv"

labeled = pd.read_csv(os.path.join(DATA_PATH, labeled))
#sim_labels = pd.read_csv(os.path.join(DATA_PATH, sim_labels))
comment_length=10
with open(os.path.join(DATA_PATH,processed_file),"r", encoding="utf-8") as f:
    processed_docs = f.readlines()
length_include = [len(re.sub("\d+", "", x.strip()).split(','))>5 for x in processed_docs]

comments = pd.read_csv(os.path.join(DATA_PATH, "merged_comments.csv"))
#comments = comments[comments.comment_text.notnull()].copy()

print(f"the total number of comments is {comments.shape[0]+labeled.shape[0]}")

comments2 = pd.DataFrame(data={'comment_text':comments.comment_text.tolist()+labeled.comment_text.tolist()})
comments2 = comments2[comments2.comment_text.notnull()].copy()
comments2['include'] = length_include
comments2['processed'] = processed_docs
print(f"the total number of comments processed is {comments2[comments2.include].shape[0]}")

the total number of comments is 830579
the total number of comments processed is 414616


## Loading the Model

In [317]:
# model_name = "top2vec_lowercase_learn_doc2vec.model"
# model_name = "top2vec_lowercase_newpreprocessed_deep-learn_universal-sentence-encoder.model"
model_name = "top2vec_lowercase_newpreprocessed2_deep-learn_universal-sentence-encoder-multilingual-large.model"
# model_name = "top2vec_lowercase_newpreprocessed2_deep-learn_doc2vec.model"
model = Top2Vec.load(f"E:/{model_name}")

In [410]:
model.document_ids

array([     0,      1,      2, ..., 414613, 414614, 414615], dtype=int64)

### Topic Modifications
Refine topics only for model: 
top2vec_lowercase_newpreprocessed2_deep-learn_universal-sentence-encoder-multilingual-large.model

In [541]:
modified_topics = pd.read_csv(os.path.join(DATA_PATH, "topics_pruned.csv"))
print(modified_topics.shape)
print(modified_topics.head())

(727, 52)
   to_be_removed  topic_num  topic1    topic2         topic3     topic4  \
0            NaN          0    yoga    yogatx  sarahbethyoga  gymnastic   
1            NaN          1   today  sciatica            hoy      squat   
2            NaN          2  nicole   nicolle     pleasantly    fitness   
3            NaN          3    yoga    yogatx          jyada      yogic   
4            NaN          4    yoga    yogatx          yogic       yogi   

       topic5     topic6         topic7      topic8  ...      topic41  \
0  meditation      yogic         pilate  meditative  ...          joy   
1   streching     pilate           sore      tendon  ...         yoga   
2     workout     lovely       thankful      gladly  ...   gracefully   
3         day       hari           yogi     january  ...  goodmorning   
4       jyada  gymnastic  sarahbethyoga  meditation  ...     julianna   

      topic42   topic43    topic44     topic45   topic46    topic47  topic48  \
0    sciatica      j

In [547]:
# Identify the topics to be removed
topics_to_remove = modified_topics[modified_topics.to_be_removed==1].topic_num.tolist()
print(f"the number of topics to be removed is {len(topics_to_remove)}")

# Get the top topics for each document
topic_nums, topic_scores, topic_words, word_scores = model.get_documents_topics(doc_ids=model.document_ids)
print(topic_nums.shape)
print(topic_nums[:10])

# Identify the documents to be removed
docs_to_remove = []
for i, doc_top in enumerate(topic_nums):
    # if the number of topics returned is more than 1, then use doc_top[0]
    if doc_top in topics_to_remove:
        docs_to_remove.append(i)
print(f"the number of documents to be removed is {len(docs_to_remove)}")

the number of topics to be removed is 181
(414616,)
[ 39  61  52 429 128 222  38 302  11 128]
the number of documents to be removed is 52601


In [310]:
# Test the model with queries
queries = {}
queries['query1'] = "I want to lose weight"
queries['query2'] = "I want to be healthy"
queries['query3'] = "I love the yoga community"
queries['query4'] = "I want to be fit"
queries['query5'] = "I want to ease my anxiety"
queries['query6'] = "I want to look good"
queries['query7'] = "I love doing Yoga with other people"

In [318]:
print(f"model: {model_name}")
for k, query in queries.items():
    print(f"query: {query}")
    docus, scores, doc_ids = model.query_documents(query, num_docs=5)
    print(docus)

model: top2vec_lowercase_newpreprocessed2_deep-learn_universal-sentence-encoder-multilingual-large.model
query: I want to lose weight
['want lose weight plz help sir' 'want lose kg lose kg help'
 'want lose weight low body plz help' 'mera weight kg want lose weight'
 'hi bro want lose weight m try not m want weight']
query: I want to be healthy
['want feel good body healthy habit'
 'shoutout want good healthy change body live'
 'wish healthy life fill health joy light'
 'good k wish love health happiness'
 'want well physical mental health acceptance']
query: I love the yoga community
['love yoga yoga luna yoga community thank' 'yay love s yoga s awesome'
 'thankful beautiful yoga community awesome got'
 'absolute fav love yoga cz thank'
 'thank dedication yoga community pleasure practice namaste']
query: I want to be fit
['want feel good body healthy habit'
 'omg want body like haha tell exercise'
 'want well posture exercise slouch want exercise'
 'want lose weight low body plz help'

### Document ID conversion

In [413]:
# converting the old index to the new index
comments2['doc_id'] = np.NaN
comments2.loc[comments2.include,'doc_id'] = np.arange(comments2.include.sum())
print(comments2.head())
comments2.doc_id.describe()
id_conv = comments2.doc_id.to_dict()
for k,v in id_conv.items():
    if np.isnan(v):
        id_conv[k] = None
    else:
        id_conv[k] = int(v)

# convert an array of ids to the new index (if the id is not in the new index, it will be None)
def convert_id_l2s(x):
    # Long version to short version
    return [id_conv.get(i) for i in x if id_conv.get(i) is not None]

# convert new ids to old ids
id_conv2 = comments2[comments2.include].doc_id.to_dict()
id_conv2 = {int(v):k for k,v in id_conv2.items() if v is not None}

def convert_id_s2l(x):
    # Short version to long version
    return [id_conv2.get(i) for i in x]

                                        comment_text  include  \
0  I loved doing this video with my dad I think i...     True   
1             bre is very good at yoga same with Flo    False   
2  Of course I can add 15 more minutes to round o...     True   
3  Thanks for helping round out my routine, was q...     True   
4          First thing tomorrow morning! Thank you 🙏    False   

                                           processed  doc_id  
0                    loved,video,dad,think,th,good\n     0.0  
1                                bre,good,yoga,flo\n     NaN  
2  course,add,minute,round,morning,practice,pulse...     1.0  
3   thank,help,round,routine,stagnant,find,channel\n     2.0  
4                     thing,tomorrow,morning,thank\n     NaN  


In [414]:
# test the conversion
print(convert_id_l2s([0,1,2,3,4,5,6,7,8,9,10]))
print(convert_id_s2l([0,1,2,3,4,5,6,7,8,9,10]))

[0, 1, 2, 3]
[0, 2, 3, 7, 12, 16, 17, 19, 25, 27, 29]


In [548]:
# convert documnet ids for the removed documents
docs_to_remove_old = convert_id_s2l(docs_to_remove)
print(docs_to_remove_old[:10])

[62, 86, 133, 142, 144, 146, 154, 156, 164, 171]


## Label using survey documents

In [354]:
Enjoyment = ["it is fun", "i like to do yoga", "it makes me happy", "it is interesting", "i enjoy yoga", "i find yoga stimulating", "i like the excitement of participation"]
Competence = ["i like engaging in yoga which physically challenge me", "i want to obtain new skills", "i want to improve existing skills", "i like the challenge", "i want to keep up my current skill level", "i like activities which are physically challenging to me", "i want to get better at yoga"]
Appearance = ["i want to look better", "i want to improve my appearance", "i want to be attractive", "i want to improve my body shape"] #"i feel physically unattractive if i do not do yoga"
Fitness = ["i want to be physically fit", "i want to have more energy", "i want to improve my cardiovascular fitness", "i want to maintain my physicall strength to live a healthy life", "i want to maintain my physical health and well-being"]
Social = ["i love the community", "i want to be with my friends", "i like to be with others who are interested in this activity", "i want to meet new people", "my friends want me to", "i enjoy spending time with others doing yoga"]

In [444]:
def get_scores(queries, comment_id = True):
    """
    This function takes a list of queries and returns a dictionary of scores for each query
    Parameters:
    queries (list): a list of queries

    comment_id (bool): if True, the keys of the dictionary are comment ids, otherwise they are model doc ids

    Returns:
    scores (dict): a dictionary of average scores for each query with ids as keys
    """
    scores = {}
    num_queries = len(queries)
    for query in queries:
        docus, scores, doc_ids = model.query_documents(query, num_docs=100000, ef=400000)
        scores_dict = {}
        if comment_id:
            doc_ids = convert_id_s2l(doc_ids)
        for score, doc in zip(scores, doc_ids):
            if doc in scores:
                scores_dict[doc] += score/num_queries
            else:
                scores_dict[doc] = score/num_queries
    return scores_dict

In [445]:
# Get scores for each label
label_scores = {}
label_scores['Enjoyment'] = get_scores(Enjoyment)
label_scores['Competence'] = get_scores(Competence)
label_scores['Appearance'] = get_scores(Appearance)
label_scores['Fitness'] = get_scores(Fitness)
label_scores['Social'] = get_scores(Social)

In [446]:
# propgate the scores to the comments
comments_df = pd.DataFrame(data={'comment_text': comments.comment_text.tolist() + labeled.comment_text.tolist()})
comments_df = comments_df[comments_df.comment_text.notnull()].copy()
comments_df['include'] = length_include
comments_df['processed'] = processed_docs

# fill the column for each label with the similarity scores using the doc ids
for label, scores in label_scores.items():
    print(label, len(scores))
    comments_df[label] = 0.0
    comments_df.loc[list(scores.keys()), label] = list(scores.values())
    
main_cols = ['Competence', 'Fitness', 'Appearance', 'Enjoyment', 'Social']

# get the max score for each comment
comments_df['max_score'] = comments_df[main_cols].max(axis=1)
# get the label for each comment
comments_df['label'] = comments_df[main_cols ].idxmax(axis=1)

pprint(comments_df[main_cols].describe())


Enjoyment 100000
Competence 100000
Appearance 100000
Fitness 100000
Social 100000
          Competence        Fitness     Appearance      Enjoyment  \
count  830479.000000  830479.000000  830479.000000  830479.000000   
mean        0.006620       0.005541       0.007764       0.004531   
std         0.018354       0.015347       0.021463       0.012433   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0.000000   
max         0.113910       0.146740       0.142680       0.089487   

              Social  
count  830479.000000  
mean        0.007589  
std         0.021007  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         0.000000  
max         0.120060  


In [448]:
cut_off = 0.08
#comments_df.loc[comments_df.max_score < cut_off, 'label'] = 'other'
pprint(comments_df[comments_df.max_score > cut_off].label.value_counts())

label
Appearance    9569
Social        8074
Fitness        800
Competence     558
Enjoyment        4
Name: count, dtype: int64


In [478]:
def print_top_label(label,top=20, print_text=True):
    for k,row in comments_df[comments_df.label==label].sort_values(by=label, ascending=False).iloc[:top].iterrows():
        print(f"{row.label} ({row[row.label]:.2f}):")
        if print_text:
            pprint(row.comment_text)
        else:
            pprint(row.processed)
        print('------------------')

# sample social labels
label = "Appearance"
print_top_label(label, top=10)

Appearance (0.49):
'Never seen abs shaped like that. How did the gap in between come there'
------------------
Appearance (0.45):
'what beautiful graceful forms and shapes. absolutely lovely!'
------------------
Appearance (0.44):
'I am so out of shape, lol.. I did it but it was hard! I love it... thank ypu'
------------------
Appearance (0.43):
'A M A Z I N G Allie, more like these please! This was everything and more.'
------------------
Appearance (0.42):
('you look so much better in 2022. Still very slim and much more toned but not '
 'way too thin.')
------------------
Appearance (0.42):
('I always missed you.You illustrated very lucidly & charming manners. You are '
 'very attractive & lovely.')
------------------
Appearance (0.41):
'Kya sach me breast shape me aa jayega'
------------------
Appearance (0.41):
('Oh I LOVE this!!! I’ve always been an all or nothing person and this is such '
 'a beautiful way of looking at it')
------------------
Appearance (0.40):
"This looks incre

## Documents by Keywords

In [549]:
keywords = {}
keywords['Enjoyment'] = ["fun", "happy", "interesting", "excitement"] # "stimulating"
keywords['Competence'] = ["challenge", "skill", "improve", "learn"]
keywords['Appearance'] = ["look", "attractive", "shape"]
keywords['Fitness'] = ["fit", "energy", "cardio", "strength", "health"]
keywords['Social'] = ["community", "friend", "people"] # "others"

In [551]:
key_scores = {}
for label, keys in keywords.items():
    print(label)
    docus, scores, doc_ids = model.search_documents_by_keywords(keys, num_docs=20000, ef=400000)
    doc_ids = convert_id_s2l(doc_ids)
    key_scores[label] = {doc_id:score for doc_id, score in zip(doc_ids, scores)}
    for key in keys:
        print(f"{comments_df[comments_df.comment_text.str.contains(key)].shape[0]} of {key}")
    print('------------------')

Enjoyment
9985 of fun
12682 of happy
1654 of interesting
143 of excitement
------------------
Competence
22664 of challenge
578 of skill
3268 of improve
6984 of learn
------------------
Appearance
19424 of look
94 of attractive
1434 of shape
------------------
Fitness
6552 of fit
9497 of energy
1252 of cardio
10924 of strength
6605 of health
------------------
Social
4983 of community
6535 of friend
8368 of people
------------------


In [552]:
# propgate the scores to the comments
# comments_df = pd.DataFrame(data={'comment_text': comments.comment_text.tolist() + labeled.comment_text.tolist()})
# comments_df = comments_df[comments_df.comment_text.notnull()].copy()
# comments_df['include'] = length_include
# comments_df['processed'] = processed_docs

# fill the column for each label with the similarity scores using the doc ids
for label, scores in key_scores.items():
    print(label, len(scores))
    comments_df[label] = 0.0
    comments_df.loc[list(scores.keys()), label] = list(scores.values())
    
main_cols = ['Competence', 'Fitness', 'Appearance', 'Enjoyment', 'Social']

# remove the documents that are to be removed
comments_df.loc[docs_to_remove_old, main_cols] = 0.0
#TODO: remove the documents in other methods as well (query, keywords, etc.)

# get the max score for each comment
comments_df['max_score'] = comments_df[main_cols].max(axis=1)
# get the label for each comment
comments_df['label'] = comments_df[main_cols ].idxmax(axis=1)

pprint(comments_df[main_cols].describe())

Enjoyment 20000
Competence 20000
Appearance 20000
Fitness 20000
Social 20000
          Competence        Fitness     Appearance      Enjoyment  \
count  830479.000000  830479.000000  830479.000000  830479.000000   
mean        0.005229       0.006572       0.004339       0.006410   
std         0.034644       0.042796       0.029695       0.043817   
min         0.000000       0.000000       0.000000       0.000000   
25%         0.000000       0.000000       0.000000       0.000000   
50%         0.000000       0.000000       0.000000       0.000000   
75%         0.000000       0.000000       0.000000       0.000000   
max         0.453253       0.572973       0.489017       0.568918   

              Social  
count  830479.000000  
mean        0.003316  
std         0.024484  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         0.000000  
max         0.451608  


In [553]:
cut_off = 0.3
comments_df.loc[comments_df.max_score < cut_off, 'label'] = 'other'
pprint(comments_df[comments_df.max_score > cut_off].label.value_counts())

label
Enjoyment     6546
Fitness       4053
Competence     594
Appearance     209
Social         135
Name: count, dtype: int64


In [555]:
print_top_label('Appearance', top=10, print_text=False)

Appearance (0.49):
'see,ab,shape,like,gap,come\n'
------------------
Appearance (0.45):
'beautiful,graceful,form,shape,absolutely,lovely\n'
------------------
Appearance (0.44):
'shape,lol,hard,love,thank,ypu\n'
------------------
Appearance (0.42):
'look,well,slim,toned,way,thin\n'
------------------
Appearance (0.42):
'miss,illustrate,lucidly,charming,manner,attractive,lovely\n'
------------------
Appearance (0.41):
'kya,sach,breast,shape,aa,jayega\n'
------------------
Appearance (0.41):
'oh,love,person,beautiful,way,look\n'
------------------
Appearance (0.40):
'look,incredible,m,look,forward,try\n'
------------------
Appearance (0.40):
'look,cute,thank,positiv,make,positiv\n'
------------------
Appearance (0.39):
'lovey,flow,mind,ask,get,look,like\n'
------------------


In [567]:
# sample data 100 rows of each label
comments_df['doc_id'] = comments_df.index
sample_size = 100
sample_data = []
for label in main_cols:
    sample_data = sample_data + comments_df[comments_df.label==label].sample(sample_size).values.tolist()

sample_df = pd.DataFrame(data = sample_data, columns = comments_df.columns)
print(sample_df.shape)
sample_df.to_csv(os.path.join(DATA_PATH, "comments_keywords_labels_sample_010324.csv"), index=True)

#comments_df['rand'] = np.random.rand(comments_df.shape[0])
#comments_df.to_csv(os.path.join(DATA_PATH, "comments_keywords_labels_010324.csv"), index=True)

(500, 12)


In [571]:
# checking the accuracy of the labels

# first loading the labeled data
labeled_df = pd.read_csv(os.path.join(DATA_PATH, "comments_keywords_labels_sample_010324.csv"))
print(labeled_df.shape)
print(labeled_df.isnull().sum())
labeled_df = labeled_df[labeled_df.true_label.notnull()].copy()
print(labeled_df.shape)

(500, 13)
Unnamed: 0       0
comment_text     0
include          0
processed        0
Enjoyment        0
Competence       0
Appearance       0
Fitness          0
Social           0
max_score        0
label            0
doc_id           0
true_label      25
dtype: int64
(475, 13)


In [576]:
# get the accuracy of the labels
# calculate the precision and recall for each label
label_acc_scores = {}
for label in main_cols:
    print(label)
    label_acc_scores[label] = {}
    label_acc_scores[label]['precision'] = labeled_df[(labeled_df.true_label==label) & (labeled_df.label==label)].shape[0] / labeled_df[labeled_df.label==label].shape[0]
    label_acc_scores[label]['recall'] = labeled_df[(labeled_df.true_label==label) & (labeled_df.label==label)].shape[0] / labeled_df[labeled_df.true_label==label].shape[0]
    print(label_acc_scores[label])
    print('------------------')

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

mask = labeled_df.true_label != 'other'
print(f1_score(labeled_df[mask].true_label.values, labeled_df[mask].label.values, average="macro"))
print(precision_score(labeled_df[mask].true_label.values, labeled_df[mask].label.values, average="macro"))
print(recall_score(labeled_df[mask].true_label.values, labeled_df[mask].label.values, average="macro"))

Competence
{'precision': 0.8762886597938144, 'recall': 0.9340659340659341}
------------------
Fitness
{'precision': 0.9120879120879121, 'recall': 0.9431818181818182}
------------------
Appearance
{'precision': 0.6344086021505376, 'recall': 1.0}
------------------
Enjoyment
{'precision': 0.9696969696969697, 'recall': 0.9320388349514563}
------------------
Social
{'precision': 0.8631578947368421, 'recall': 0.9761904761904762}
------------------
0.6768687703229308
0.6716236291046724
0.6836395804842407


c:\Users\doosti\.conda\envs\top2vec\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  return result


In [577]:
# Saving the results
comments_df.pop('')

,comment_text,include,processed,Enjoyment,Competence,Appearance,Fitness,Social,max_score,label,rand,doc_id
0,I loved doing this video with my dad I think i...,True,"loved,video,dad,think,th,good\n",0.0,0.0,0.0,0.0,0.0,0.0,other,0.760410,0
1,bre is very good at yoga same with Flo,False,"bre,good,yoga,flo\n",0.0,0.0,0.0,0.0,0.0,0.0,other,0.075107,1
2,Of course I can add 15 more minutes to round o...,True,"course,add,minute,round,morning,practice,pulse...",0.0,0.0,0.0,0.0,0.0,0.0,other,0.639350,2
3,"Thanks for helping round out my routine, was q...",True,"thank,help,round,routine,stagnant,find,channel\n",0.0,0.0,0.0,0.0,0.0,0.0,other,0.689490,3
4,First thing tomorrow morning! Thank you 🙏,False,"thing,tomorrow,morning,thank\n",0.0,0.0,0.0,0.0,0.0,0.0,other,0.174636,4


In [499]:
topic_words, topic_scores, topic_nums = model.get_topics(591)

In [501]:
topic_words[590]

array(['frm', 'ie', 'hm', 'mmm', 'af', 'bs', 'bt', 'sm', 'ei', 'mm', 'nd',
       'hiii', 'wt', 'ytt', 'tk', 'ca', 'bta', 'fo', 'jk', 'ai', 'nt',
       'aw', 'yt', 'th', 'tks', 'tq', 'thnk', 'hv', 'ce', 'hmmm', 'fwfg',
       'rn', 'btao', 'sa', 'ye', 'ho', 'hmm', 'ngl', 'tnx', 'yr', 'bp',
       'fr', 'bcz', 'lo', 'aa', 'ta', 'ms', 'bht', 'hr', 'je'],
      dtype='<U15')

In [538]:
topic_words, word_scores, topic_nums = model.get_topics()
topics_df = pd.DataFrame(data = topic_words, columns=[f"topic{i+1}" for i in range(50)])
topics_df.index = topic_nums
topics_df.to_csv(os.path.join(DATA_PATH, "topics.csv"))

## Finding Similarities

In [319]:
pprint(labeled.columns)

Index(['channel_name', 'comment_text', 'habit', 'community', 'progress',
       'Fitness', 'Competence', 'Appearance', 'Enjoyment', 'Social',
       'predicted_label', 'label', 'included'],
      dtype='object')


In [324]:
label_idx = {}
# Competence
comp_idx = labeled[labeled.Competence == 1].index.values + comments.shape[0]
label_idx['competence'] = comp_idx.tolist()
# Fitness
fit_idx = labeled[labeled.Fitness == 1].index.values + comments.shape[0]
label_idx['fitness'] = fit_idx.tolist()
# Appearance
app_idx = labeled[labeled.Appearance == 1].index.values + comments.shape[0]
label_idx['appearance'] = app_idx.tolist()
# Enjoyment
enj_idx = labeled[labeled.Enjoyment == 1].index.values + comments.shape[0]
label_idx['enjoyment'] = enj_idx.tolist()
# Social
#soc_idx = labeled[(labeled.Social == 1)].index.values + comments.shape[0]
#label_idx['social'] = soc_idx.tolist()

## Extra categories
# Appreciation
# label_idx['thanks'] = [325, 1086, 9474, 4650, 7789, 1036, 358,
#                        223814, 316898, 610256, 301454, 212997, 356964, 223083, 294452, 59093, 344656,
#                        315086, 344725, 44930]
# More
# label_idx['more'] = [300363, 371792, 628093, 301235, 44240]
# # Great
# label_idx['great'] = [86589,
#                       102915, 347408, 580749, 103456, 314672, 44240]
# Journey
#label_idx['journey'] = [216508, 217844]

In [275]:
comments_df = pd.DataFrame(data={'comment_text': comments.comment_text.tolist() + labeled.comment_text.tolist()})
comments_df['include'] = length_include
comments_df['processed'] = processed_docs
print(comments_df.shape)

# get similarity scores for each comment
def get_similarity_scores(idx, model, num_docs=100000):
    docs, scores, doc_ids = model.search_documents_by_documents(convert_id_l2s(idx), num_docs=num_docs, ef=400000)
    return scores, convert_id_s2l(doc_ids)

# fill the column for each label with the similarity scores using the doc ids
def fill_column(idx, model, label):
    scores, doc_ids = get_similarity_scores(idx, model)
    comments_df[label] = 0.0
    comments_df.loc[doc_ids, label] = scores

# fill the columns for each label
for label, idx in label_idx.items():
    fill_column(idx, model, label)

# extra for social
soc_idx = (labeled[(labeled.Social == 1)].index.values + comments.shape[0]).tolist()
soc_neg = [2+comments.shape[0], 344659]#, 216508, 344659, 212698, 307772, 301844, 212609, 317443, 215525, 67716, 810299,
           #217844, 102735, 102912, 103336, 16240, 304445,322119,160198] 

docs, scores, doc_ids = model.search_documents_by_documents(doc_ids = convert_id_l2s(soc_idx), doc_ids_neg = convert_id_l2s(soc_neg), num_docs=100000)
comments_df['social'] = 0.0
comments_df.loc[convert_id_s2l(doc_ids), 'social'] = scores

main_cols = ['competence', 'fitness', 'appearance', 'enjoyment', 'social']
extra_cols = [] #['thanks', 'more', 'great', 'journey']

# get the max score for each comment
comments_df['max_score'] = comments_df[main_cols + extra_cols].max(axis=1)
# get the label for each comment
comments_df['label'] = comments_df[main_cols + extra_cols].idxmax(axis=1)

pprint(comments_df.sample(10))


(830479, 3)
                                             comment_text  include  \
321152  Why were my thighs shaking uncontrollably in a...     True   
829509  My word of the day is strong.  It‚Äôs my birth...     True   
24261   Is this flow save if you have 1 finger gap dia...     True   
173481  Thank u!  I needed some TLC!  Thank u!  🧘‍♀️🧘‍...     True   
681035  I am suffering from pcod and I want to loss 15...    False   
238205  I always find day 30 hard without Adriene's re...     True   
162908                     I needed this today; thank you    False   
414752  Thank you. That felt wonderful. Do so love the...     True   
377868        and the funny thing is they never worked...    False   
614101  Learn Yoga & Dance...\n\nhttps://youtube.com/s...    False   

                                                processed  competence  \
321152      thigh,shake,uncontrollably,svasana,pose,end\n    0.000000   
829509  word,day,strong,birthday,set,stage,trip,sun,na...    0.000000  

In [283]:
cut_off = 0.6
comments_df.loc[comments_df.max_score < cut_off, 'label'] = 'other'

In [282]:
pprint(comments_df[comments_df.max_score > cut_off].label.value_counts())

label
enjoyment     27684
competence    17924
social        14611
fitness        7771
appearance        1
Name: count, dtype: int64


In [284]:
for k,row in comments_df[comments_df.label=="social"].sort_values(by='social', ascending=False).iloc[:20].iterrows():
# print social score with two decimal places
    print(f"score:{row.social:.2f}, index:{k}, label index:{k-comments.shape[0]}")
    pprint(row.comment_text)
    # pprint(row.processed)
    # print(length_include[k])
    print('------------------')

score:0.79, index:829245, label index:1765
("It's May, and I started this in January, but I'm still so proud of myself "
 'for finishing and learning to love myself along the way! Thanks for the '
 'great yoga journey Adriene!')
------------------
score:0.78, index:235090, label index:-592390
("Thank you Adriene. I'm feeling so strong, and greatful after todays "
 'practice. Love you. 😌 Thank you yoga community. Have a great rest of the '
 'day.')
------------------
score:0.78, index:180458, label index:-647022
('Amazing. I love this flow! Wonderful practice today.  I am just so loving '
 'the daily yoga, I can feel myself looking forward to the continued journey '
 'after Jan. Thank you, Adriene!')
------------------
score:0.78, index:170043, label index:-657437
('Thank you so much Adriene for sharing this amazing times for whole 30 days '
 'again. I have started yoga with you in Jan 19. \n'
 'I love to find myself and how I feel here on the mat with you and everyone '
 'else joining 

In [22]:
pprint(soc_neg)

[221981, 131228, 62165, 224507, 191522, 207289, 224867, 86560, 73700]


In [285]:
comments2[(comments2.comment_text.str.contains("thank")) & (comments2.include)]

,comment_text,include,doc_id
16,Yesterday I was searching for a morning class ...,True,5.0
55,This is one of my favourites! I put it in the ...,True,25.0
73,thank you for always being so present in your ...,True,32.0
79,I was feeling so unmotivated to do my practice...,True,33.0
87,So many morning I started with this routine an...,True,38.0
...,...,...,...
830455,"When you said ""whisper to yourself I am strong...",True,425484.0
830462,This was my favourite of all the 30 Day videos...,True,425488.0
830464,Hi! I knw this is so random and unrelated but ...,True,425490.0
830469,this video really changed my body flow...\nMan...,True,425493.0


In [556]:
# compare the labels to the original labels
labeled['predicted_label'] = comments_df.loc[comments.shape[0]:, 'label'].tolist()
labeled['included'] = comments_df.loc[comments.shape[0]:, 'include'].tolist()

In [557]:
labeled['label'] = labeled.loc[:,['Fitness', 'Competence', 'Appearance', 'Enjoyment', 'Social']].idxmax(axis=1)
labeled.loc[labeled.loc[:,['Fitness', 'Competence', 'Appearance', 'Enjoyment', 'Social']].max(axis=1)<0.5,'label'] = 'other'

#pprint(labeled.label.value_counts())
#pprint(labeled.predicted_label.value_counts())
labeled_included = labeled[labeled.included]

for label in labeled_included.label.unique():
    print(label, labeled_included[labeled_included.label==label].shape[0])
    print(labeled_included[labeled_included.label==label].predicted_label.value_counts())
    print()

Competence 32
predicted_label
other    32
Name: count, dtype: int64

Enjoyment 18
predicted_label
other      17
Fitness     1
Name: count, dtype: int64

other 1917
predicted_label
other         1883
Enjoyment       22
Fitness          8
Competence       3
Appearance       1
Name: count, dtype: int64

Appearance 3
predicted_label
other    3
Name: count, dtype: int64

Fitness 20
predicted_label
other    20
Name: count, dtype: int64

Social 20
predicted_label
other    20
Name: count, dtype: int64



In [298]:
comments_df.loc[list(labeled_included[labeled_included.label=="Social"].index.values + comments.shape[0])]

,comment_text,include,processed,competence,fitness,appearance,enjoyment,social,max_score,label
827516,Practiced with more than 400 people today!! Lo...,True,"practice,people,today,lovely,vinyasa,expand,da...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,other
827633,Hi Tim! I also will love to keep up with the c...,True,"hi,tim,love,challenge,quarantine,bring,sense,c...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,other
827637,I'm glad I could complete this class in the fi...,True,"m,glad,complete,class,hour,upload,incredible,r...",0.584763,0.540039,0.000000,0.000000,0.0,0.584763,other
827671,I love this community an I'm so proud to be a ...,True,"love,community,m,proud,thank,adriene,help,find...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,other
827678,love this! another long one for courage 2020.....,True,"love,long,courage,gather,body,happy,mind,rest,...",0.605427,0.637968,0.000000,0.656868,0.0,0.656868,enjoyment
827748,Just hit the refresh button after the practice...,True,"hit,refresh,button,practice,number,view,go,gue...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,other
827882,The comment section is such a safe space... ev...,True,"comment,section,safe,space,day,love,energy,lov...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,other
827900,Yell if you're coming straight from HOME! Nama...,True,"yell,come,straight,home,namaste,friend,let,con...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,other
827940,"‚ÄúTears are words that need to be shed‚Äù, Pa...",True,"äútear,word,need,paul,cohelho,tear,flow,eye,to...",0.000000,0.522600,0.000000,0.000000,0.0,0.522600,other
827974,"""Your precious life, your precious body ..."" A...",True,"precious,life,precious,body,say,move,shift,fee...",0.000000,0.000000,0.000000,0.000000,0.0,0.000000,other


In [299]:
docs, scores, doc_ids = model.search_documents_by_documents(doc_ids = convert_id_l2s([827671]), num_docs=10, ef=400000)
print(convert_id_s2l(doc_ids))

[515619, 223728, 263519, 169274, 268895, 244393, 280170, 163723, 170568, 166280]


In [304]:
model.query_documents(query = "i love this community i appreciate everyone", num_docs=10, return_documents=True, ef=400000)

(array(['nice community mean lot thank guy',
        'love thank congrat m member beautiful community almost ',
        'm forever grateful community love tomorrow',
        'grateful m keep go give pride love community',
        'extremely grateful community thank put fun day  days',
        'thank adriene thank community send love',
        'definitely love know hard time lot gratitude community share thank',
        'thank dear adriene m glad continual support namaste community',
        'love community help immensely want thank true',
        'thank adriene lot love beautiful community glad share space'],
       dtype=object),
 array([0.7862143 , 0.7063819 , 0.7062247 , 0.7010541 , 0.6820182 ,
        0.6601259 , 0.6581006 , 0.65275234, 0.65250397, 0.6490565 ],
       dtype=float32),
 array([160753, 124158, 147512, 155127,  54461, 125965, 280602,  82565,
        117194, 100663], dtype=int64))